In [1]:
from datetime import datetime
import os
import sys
import time

from datasets import load_dataset
import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/opt/homebrew/anaconda3/envs/conda-venv/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
base_model = "codellama/CodeLlama-7b-python-hf"
orig_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype = torch.float16,
    device_map = "auto",
    offload_folder = "./CodeLlama-7b-python-hf/model/",
    offload_state_dict = True
)
tokenizer = AutoTokenizer.from_pretrained(base_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
data = load_dataset("csv", data_files={"train": ["./dataset/dataset.csv"], "validation": "./dataset/dataset.csv", "test": "./dataset/dataset.csv"})

In [4]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = f'Requirements: \n\n{data_point["input_ids"]}\n\nSummary: '
    return tokenize(full_prompt)

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = data.map(generate_and_tokenize_prompt)
print(tokenized_datasets)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 5
    })
    validation: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 5
    })
})


In [5]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

model = prepare_model_for_int8_training(orig_model)
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(model, config)

In [6]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

peft_model = get_peft_model(orig_model, config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 16777216
all model parameters: 6755192832
percentage of trainable model parameters: 0.25%


In [10]:
output_dir = f"./peft-dialogue-summary-training-{str(int(time.time()))}"

# Set Training parameters
batch_size = 128
per_device_train_batch_size = 32
gradient_accumulation_steps = batch_size // per_device_train_batch_size

training_args = TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        max_steps=400,
        learning_rate=3e-4,
        logging_steps=10,
        optim="adamw_torch",
        evaluation_strategy="no", # if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=20,
        save_steps=20,
        output_dir=output_dir,
        load_best_model_at_end=False,
        group_by_length=True, # group sequences of roughly the same length together to speed up training
        report_to="none", # if use_wandb else "none",
        run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wandb else None,
    )

trainer = Trainer(
    model=peft_model,
    train_dataset=tokenized_datasets["train"],
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
trainer.train()

Step,Training Loss
10,1.105600
20,1.024400
30,0.778500
40,0.354200
50,0.066000
60,0.022600
70,0.020600
80,0.020100
90,0.020000
100,0.020000
